# PART 2 - NER SERVICE WITH FASTAPI & DOCKER

For the second part of the project, we are going to create a service that receives NER requests and outputs the extracted entities from within the text sent, with their labels and confidence scores.
We will divide this part in two steps:
1) Creating an architecture that allows users to send text to an API and receive the extracted entities and their class - we will achieve this using a FastAPI architecture
2) Building a Docker container to run this service

## 1. Fast API Architecture - defining and running

We will write a brief script implementing a FastAPI architecture. I am copying the code here below in order to better organize the Jupyter Notebook, but I also have a .py file inside this folder with the same code, in order to call it from the terminal when we'll run the service and the Docker container.

Defining imports

In [1]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
import torch
import numpy as np

Defining FastAPI code

In [2]:
# Input model
class TextRequest(BaseModel):
    text: str


# Initializing
app = FastAPI()

# Loading the pre-trained model and tokenizer from Hugging Face
MODEL_NAME = "CyberPeace-Institute/SecureBERT-NER"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForTokenClassification.from_pretrained(MODEL_NAME)

device = 0 if torch.cuda.is_available() else -1  # -1 == CPU

ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="first", device=device)


@app.post("/ner")
async def extract_entities(text_request: TextRequest):
    """Extract named entities from text."""
    
    text = text_request.text
    if not text:
        raise HTTPException(status_code=400, detail="Text must be provided.")

    try:

        entities = ner_pipeline(text)

        # Converting numpy types to native types - for JSON
        for entity in entities:
            if isinstance(entity.get('score'), np.float32):
                entity['score'] = float(entity['score'])

        return {"entities": entities}

    except Exception as e:
        # Error handling
        print(f"Error during NER processing: {e}")
        raise HTTPException(status_code=500, detail="Internal Server Error during NER processing.")
    
    

After defining the FastAPI architecture, we're going to run it with a terminal command.

Note: we will use the prefix ! to run shell commands in Jupyter Notebook

In [36]:
!uvicorn ner_service_fastapi:app --host localhost --port 8000

INFO:     Started server process [52309]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://localhost:8000 (Press CTRL+C to quit)
^C
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [52309]


The FastAPI service is up and running!

We can, for example, go to the docs URL to see that the service is up:
http://localhost:8000/docs

We will want to stop this cell from running before the next step, as we'll want to run some more shell commands and we don't want it to block them 

## 2. Docker Container - defining a Dockerfile, building and running

Next, we are going to define a dockerfile, build a docker image from this dockerfile and run this docker image locally.
Note: I will assume that we have docker already installed in our environment.

Defining a Dockerfile

Here also I'll copy the code below in order to organize the Jupyter Notebook, but there is also a Dockerfile inside this folder with the same code, in order to call it from the terminal when we'll build the Docker container.

Note: The requirements.txt file is in the folder


In [ ]:
'''
# Python base image
FROM python:3.10-slim

# Setting envs so python don't buffer output
ENV PYTHONDONTWRITEBYTECODE=1
ENV PYTHONUNBUFFERED=1

WORKDIR /app

# Copying requirements.txt to dir
COPY requirements.txt /app/

# Dependencies
RUN pip install --no-cache-dir --upgrade pip && \
    pip install --no-cache-dir -r requirements.txt

# Downloading the Hugging Face model and tokenizer
RUN python -c "from transformers import AutoTokenizer, AutoModelForTokenClassification; \
                AutoTokenizer.from_pretrained('CyberPeace-Institute/SecureBERT-NER'); \
                AutoModelForTokenClassification.from_pretrained('CyberPeace-Institute/SecureBERT-NER');"

# Copying FastAPI app to dir
COPY ner_service_fastapi.py /app/

# FastApi port
EXPOSE 8000

# Run
CMD ["uvicorn", "ner_service_fastapi:app", "--host", "0.0.0.0", "--port", "8000"]
'''

Next, we'll run a shell command to build our docker image using the Dockerfile we created

Note: 
- ner_service_fastapi == tag name
- . == search for dockerfile and requirements.txt in current directory

In [3]:
! docker build -t ner_service_fastapi .

[+] Building 0.0s (0/0)  docker:desktop-linux
[+] Building 0.0s (0/0)  docker:desktop-linux
[+] Building 0.0s (0/1)                                    docker:desktop-linux
[+] Building 0.2s (1/2)                                    docker:desktop-linux
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 1.07kB                                     0.0s
 => [internal] load metadata for docker.io/library/python:3.10-slim        0.2s
[+] Building 0.4s (1/2)                                    docker:desktop-linux
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 1.07kB                                     0.0s
 => [internal] load metadata for docker.io/library/python:3.10-slim        0.3s
[+] Building 0.5s (1/2)                                    docker:desktop-linux
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfi

Optionally, we can save the docker image to a file

In [4]:
! docker save -o ner_service_image.tar ner_service_fastapi

After we built our docker image, all that we have to do is to run it! We can turn off internet access also to make sure that the container runs totally on-prem, and it still should work.

Note:
- -d == run in background mode so we can run shell code after
- -p 8000:8000 == publishes and maps 8000 host port to 8000 container port

In [6]:
! docker run -d -p 8000:8000 ner_service_fastapi

683bb0dfa5800d3b965138cae486ec75492886b543dd88c011d2e8adfad56ae9


As earlier, we can go to http://localhost:8000/docs to verify if it's running.

Lastly, we'll make a call with an example sentence to verify that we receive a correct response. We can turn off internet here also.

In [7]:
!curl -X POST http://localhost:8000/ner -H "Content-Type: application/json" -d '{"text": "Kaspersky believes both Shamoon and StoneDrill groups are aligned in their interests."}'

{"entities":[{"entity_group":"SECTEAM","score":0.9064265489578247,"word":" Kaspersky","start":0,"end":9},{"entity_group":"MAL","score":0.8394777178764343,"word":" Shamoon","start":24,"end":31},{"entity_group":"APT","score":0.9263834357261658,"word":" StoneDrill groups","start":36,"end":53}]}

We received a response with some entities, all is good!


Optional: we can stop and remove all running docker containers - but we'll not run this now because we want to use our container as part of our streamlit app!

In [40]:
!docker rm -f $(docker ps -q)

a02efea61620
